# Prediction with Nexa.
This is a very simple example that extracts the code vectors (features) from nexa and uses these to make predictions with nexa.

In [1]:
import numpy as np
import h5py
from sklearn import svm, cross_validation
from sklearn.naive_bayes import MultinomialNB

## Information extraction

In [2]:
# First we load the file 
file_location = '../results_database/text_wall_street.hdf5'
run_name = '/low-resolution'

In [3]:
# Nexa parameters
Nspatial_clusters = 5
Ntime_clusters = 15
Nembedding = 3

parameters_string = '/' + str(Nspatial_clusters)
parameters_string += '-' + str(Ntime_clusters)
parameters_string += '-' + str(Nembedding)

f = h5py.File(file_location, 'r')
nexa = f[run_name + parameters_string]

In [4]:
# Now we extract the time and the code vectors
time = nexa['time']
code_vectors = nexa['code-vectors']
code_vectors_distance = nexa['code-vectors-distance']
code_vectors_softmax = nexa['code-vectors-softmax']
code_vectors_winner = nexa['code-vectors-winner']

In [5]:
# Now we need to get the letters and align them
text_directory = '../data/wall_street_letters.npy'
letters_sequence = np.load(text_directory)
Nletters = len(letters_sequence)
symbols = set(letters_sequence)

## Predictions

In [6]:
# Parameters
N = 5000 
delay = 5

#### First let's use winner takes all

In [7]:
# Make prediction with scikit-learn
X = code_vectors_softmax[:(N-delay)]
y = letters_sequence[delay:N]
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.10)

In [8]:
clf = svm.SVC(C=1.0, cache_size=200, kernel='linear')
clf.fit(X_train, y_train)
print('SVM score', clf.score(X_test, y_test))

clf_b = MultinomialNB()
clf_b.fit(X_train, y_train)
print('Multinomial score', clf_b.score(X_test, y_test))

SVM score 0.766
Multinomial score 0.754


#### Now let's use softmax

In [9]:
# Make prediction with scikit-learn
X = code_vectors_softmax[:(N-delay)]
y = letters_sequence[delay:N]
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.10)

In [10]:
clf = svm.SVC(C=1.0, cache_size=200, kernel='linear')
clf.fit(X_train, y_train)
print('SVM score', clf.score(X_test, y_test))

clf_b = MultinomialNB()
clf_b.fit(X_train, y_train)
print('Multinomial score', clf_b.score(X_test, y_test))

SVM score 0.792
Multinomial score 0.758


#### Now we move on and use distance

#### Finally let's not use the binary representation but the normal one

In [11]:
# Make prediction with scikit-learn
X = code_vectors[:(N-delay)]
y = letters_sequence[delay:N]
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.10)

In [12]:
clf = svm.SVC(C=1.0, cache_size=200, kernel='linear')
clf.fit(X_train, y_train)
print('SVM score', clf.score(X_test, y_test))

clf_b = MultinomialNB()
clf_b.fit(X_train, y_train)
print('Multinomial score', clf_b.score(X_test, y_test))

SVM score 0.32
Multinomial score 0.238


#### Now we move on and use distance (This sometimes does not converge!, this seems dodgy)

In [13]:
# Make prediction with scikit-learn
X = code_vectors_distance[:(N-delay)]
y = letters_sequence[delay:N]
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.10)

In [14]:
clf = svm.SVC(C=1.0, cache_size=200, kernel='linear')
clf.fit(X_train, y_train)
print('SVM score', clf.score(X_test, y_test))

clf_b = MultinomialNB()
clf_b.fit(X_train, y_train)
print('Multinomial score', clf_b.score(X_test, y_test))

SVM score 0.984
Multinomial score 0.474
